In [1]:
import cirq
import qiskit as q
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate, QFT
from qiskit.synthesis import SuzukiTrotter
from openfermion import QubitOperator
from qiskit.providers.aer.library import save_statevector


import numpy as np
import scipy, math, csv, os,sys
import matplotlib.pyplot as plt

import import_ipynb
import circuit_depth_helper as cdh

importing Jupyter notebook from circuit_depth_helper.ipynb


# I. Preliminaries

Preliminary functions have been stored inside the circuit_depth_helper file

# II. Trotter Number and Circuit Error

Attempting to use the state fidelity approach to calculate Trotter number to obtain average state fidelity within a threshold of $0.4?$

In [2]:
def calc_error():
    '''
    Given a list of operators, d-levels, and qubit operator representations, 
    I construct the circuit corresponding to a second-order Suzuki-Trotter decomposition for
    iterations over Trotter steps until state fidelity is within threshold ??.
    '''
    
    # Specify the relative path to the CSV file
    relative_path = "csv_files/single_op_df_updated.csv"
    
    # Get the absolute path of the CSV file
    csv_path = os.path.abspath(relative_path)
    
    # Specify the relative path to the new CSV file with the added column
    relative_output_path = "csv_files/single_op_df_updated_v2.csv"

    # Get the absolute path of the new CSV file
    output_csv_path = os.path.abspath(relative_output_path)
    
    with open(csv_path, 'r') as infile, open(output_csv_path, 'w', newline='') as outfile:
        csv_reader = csv.reader(infile)
        csv_writer = csv.writer(outfile)
        
        # Read the header row from the original CSV file
        header = next(csv_reader)
    
        # Add the name of the new column to the header
        header.append("[Trotter number, Circuit Depth. Average Fidelity, Std. Deviation., Error]")
        
        # Write the updated header to the new CSV file
        csv_writer.writerow(header)
    
        # Iterate through each row in the CSV file
        for row in csv_reader:
            op_name = row[0]  # Access the first column (Column1)
            d_val_str = row[1]  
            d_val = list(map(int, d_val_str[1:-1].split(',')))
            enc_name = row[2]  
            qub_operator = row[3]
            locality = int(row[4])
            
            
            unary_ind=-1
            if enc_name=="Charge Encoding Unary":
                unary_ind=1
            else:
                unary_ind=0
            
            
            '''
            Iterating over Trotter step r, constructing the circuit, obtaining circuit depth, and
            calculating state fidelity statistics
            '''
            
            print(f"Beginning recursion for {op_name,d_val,enc_name}.")
            # if enc_name=='Hermite':
            #     stored_data = cdh.Hermite_Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,1)
            # else:
            #     stored_data=cdh.Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,1,0,-1,0) 
                
            stored_data=cdh.Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,1,0,-1,0) 
                
            print("(r, depth, mean fidelity, std. dev, uncertainty) = ", stored_data)
            print()
        
            # Write the updated row to the new CSV file
            row.append(stored_data)
            csv_writer.writerow(row)
            
    
    
    return

In [3]:
# calc_error()

In [4]:
def calc_fidelity():
    '''
    Given a list of operators, d-levels, and qubit operator representations, 
    I construct the circuit corresponding to a second-order Suzuki-Trotter decomposition for
    various Trotter steps and calculate average state fidelity in that circuit.
    
    The output is a large tuple in the form of 
    {[Trotter number, circuit depth, [average fidelity, std. deviation, error= (Std dev)/sample_size]]}
    '''
    
    # Specify the relative path to the CSV file
    relative_path = "csv_files/single_op_df_updated.csv"
    
    # Get the absolute path of the CSV file
    csv_path = os.path.abspath(relative_path)
    
    # Specify the relative path to the new CSV file with the added column
    relative_output_path = "csv_files/d16_single_op_df_updated_w_fidelity.csv"

    # Get the absolute path of the new CSV file
    output_csv_path = os.path.abspath(relative_output_path)
    
    
        csv_reader = csv.reader(infile)
        csv_writer = csv.writer(outfile)
        
        # Read the header row from the original CSV file
        header = next(csv_reader)
    
        # Add the name of the new column to the header
        Trotter_step_set = [1,2,3,4,6,8,10,15,20]
        for i in Trotter_step_set:
            header.append("[Trotter number, Circuit depth, Fidelity mean, std. dev, uncertainty]")
        
        # Write the updated header to the new CSV file
        csv_writer.writerow(header)
    
        # Iterate through each row in the CSV file
        for row in csv_reader:
            op_name = row[0]  # Access the first column (Column1)
            d_val_str = row[1]  
            d_val = list(map(int, d_val_str[1:-1].split(',')))
            enc_name = row[2]  
            qub_operator = row[3]
            locality = int(row[4])
            
            
            # Determining whether the op term is pure X, pure P, XP, or PX
            x_str=""
            p_str=""
            xp_str=""
            px_str=""
        
            counter0=0
            for c1_ind in range(len(op_name)):
                c=op_name[c1_ind].upper()
                if c=="X" or c=="Q":
                    counter0+=1
                    counter1=0
                    for c2_ind in range(len(op_name)):
                        E=op_name[c2_ind].upper()
                        if E=="P" and c2_ind>c1_ind:
                            xp_str=op_name
                            counter1+=1
                        elif E=="P" and c2_ind<c1_ind:
                            px_str=op_name
                            counter1+=1
                    if counter1==0:
                        x_str=op_name
                    
            if counter0==0:
                p_str=op_name
            
            unary_ind=-1
            if enc_name=="Charge Encoding Unary":
                unary_ind=1
            else:
                unary_ind=0
            
            
            
            
            
            '''
            Iterating over set of Trotter steps, constructing the circuit, and obtaining circuit depth
            '''
            
            stored_data=np.array([])# This will be the final data to go into the csv file
            # stored_data.astype(dtype=object)
            
            # Trotter_step_set = [1,10,100,1000] # The Trotter steps we want to consider
            
            if d_val[0]==16:
                for r in Trotter_step_set:
                    if x_str:
                        circ = cdh.create_circuit(op_name,qub_operator,d_val,0,r,unary_ind)
                        Circuit_depth = circ.decompose().decompose().decompose().depth()
                    elif p_str:
                        circ = cdh.create_circuit(op_name,qub_operator,d_val,1,r,unary_ind)
                        Circuit_depth = circ.decompose().decompose().decompose().depth()
                    elif xp_str:
                        # First do p:
                        circ=cdh.create_circuit(op_name,qub_operator[1],d_val,1,r,unary_ind)
                        # Next do x:
                        x_info=cdh.create_circuit(op_name,qub_operator[0],d_val,0,r,unary_ind)
                        circ.append(x_info)
                        Circuit_depth = circ.decompose().decompose().decompose().depth()
                    elif px_str:
                        # First do x:
                        circ=cdh.create_circuit(op_name,qub_operator[0],d_val,0,r,unary_ind)
                        # Next do p:
                        p_info=cdh.create_circuit(op_name,qub_operator[1],d_val,1,r,unary_ind)
                        circ.append(p_info)
                        Circuit_depth = circ.decompose().decompose().decompose().depth()
                        
                    print(f"Circuit construction complete ({enc_name}, r={r}).\nCircuit_depth for " + op_name+ ", d=", d_val, ": ", Circuit_depth)
            
            
                    
                    '''
                    Fidelity calculation:
                    
                    1. Obtain properly distributed states
                    2. Calculate exact time-evolved state
                    3. Calculate approximate time-evolved state; This is done by:
                        First initialising a circuit in the random state
                        Apply the circuit calculated above
                        Retrieve state vector afterwards
                    4. Calculate inner-product and subsequent fidelity
                    '''
                 
                    ### 1. Obtain properly distributed states ###
                    
                    # number of qubits required for the register
                    tot_qub=0
                    for register in d_val:
                        if unary_ind==0:
                            num_q = int(math.log(register,2)) 
                            tot_qub+=num_q
                        elif unary_ind==1:
                            tot_qub+=register
                    
                    # Exact time evolution unitary matrix
                    H = cdh.exact_unitary_matrix(qub_operator,d_val,unary_ind)
                    H_norm = np.linalg.norm(H, ord='fro')
                    H_t = 1/H_norm
                    # H = np.array(H_t*H)
                    
                    
                    ### Run an arbitrary number of iterations; take the mean, std. dev
                    fidelity_list=[]
                    iterations = int(1e2)
                    M = pow(2,tot_qub)
            
                    ''' Start iterations '''
                    print(f"Beginning {iterations} iterations for time-evolved states.")
                    for num_its in range(iterations):
                        psi = np.array([])
                        
                        # Constructing a random state
                        for vect_val in range(M):
                            a = np.random.normal()
                            b = np.random.normal()
                            sub_arr = np.array([a+1j*b])
                            psi = np.append(psi,sub_arr)
                        
                        
                        new_fid = cdh.fidelity(circ,H,H_t,psi,tot_qub)
                        fidelity_list = np.append(fidelity_list, new_fid)
                        
                    ''' End iterations '''
                        
                    # Calculataing statistics            
                    mean = np.real(np.mean(fidelity_list))
                    std_dev = np.std(fidelity_list)
                    uncertainty = std_dev/iterations
                    
                    data = [r,Circuit_depth, mean, std_dev, uncertainty]
                    # stored_data = np.append(stored_data, data)
            
                    print("(r, depth, mean, std. dev, uncertainty) = ",data)
                    print()
            
            # used to be here
            # row.append(stored_data)
                    # Write the updated row to the new CSV file
                    row.append(data)
                    
            csv_writer.writerow(row)
            
    return 
    

In [ ]:
calc_fidelity()


Circuit construction complete (Hermite, r=1).
Circuit_depth for q_0, d= [16] :  2
Beginning 100 iterations for time-evolved states.
(r, depth, mean, std. dev, uncertainty) =  [1, 2, 0.0, 0.0, 0.0]


Circuit construction complete (Hermite, r=2).
Circuit_depth for q_0, d= [16] :  4
Beginning 100 iterations for time-evolved states.
(r, depth, mean, std. dev, uncertainty) =  [2, 4, 0.0, 0.0, 0.0]


Circuit construction complete (Hermite, r=3).
Circuit_depth for q_0, d= [16] :  6
Beginning 100 iterations for time-evolved states.
(r, depth, mean, std. dev, uncertainty) =  [3, 6, 0.0, 0.0, 0.0]


Circuit construction complete (Hermite, r=4).
Circuit_depth for q_0, d= [16] :  8
Beginning 100 iterations for time-evolved states.
(r, depth, mean, std. dev, uncertainty) =  [4, 8, 0.0, 0.0, 0.0]


Circuit construction complete (Hermite, r=6).
Circuit_depth for q_0, d= [16] :  12
Beginning 100 iterations for time-evolved states.
(r, depth, mean, std. dev, uncertainty) =  [6, 12, 0.0, 0.0, 0.0]


Ci